In [174]:
import serial 
import struct

RETURN_TEMP   = 0x01
RETURN_SAMPLE = 0x02
RETURN_HELLO  = 0x03
SET_BIAS = 0x04
default = 0x04

serialport = '/dev/ttyUSB0'
serialport = '/dev/ttyACM0'

In [257]:
# Open serial port
with serial.Serial(serialport, 115200, timeout=10) as ser:
  #send hex command and then print the result
  ser.write(bytes([RETURN_SAMPLE, 0x00]))
  data = ser.read(8)
  if len(data) == 8:
    int1, int2 = struct.unpack('>ii', data)
    SiPM_Temp = int1 / 100.0
    SiPM_Signal = int2 / 100.0
    print("SiPM_Temp:", SiPM_Temp)
    print("SiPM_Signal:", SiPM_Signal)
  else:
    print('error')

SiPM_Temp: 1016.0
SiPM_Signal: 86.56


In [242]:
# Open serial port
with serial.Serial(serialport, 115200, timeout=1) as ser:
  #send hex command and then print the result
  ser.write(bytes([RETURN_HELLO, 0x00]))
  data = ser.read(8)
  print(data)

b'hi\r\n'


In [246]:
# Open serial port
with serial.Serial(serialport, 115200, timeout=0.5) as ser:
  #send hex command and then print the result
  ser.write(bytes([SET_BIAS, 0x0f]))
  data = ser.read(8)
  print(data)

b'_OK\r\n'
